# Importing Necessary Libraries and Modules

This first cell of the notebook imports all the necessary libraries and modules that will be used in the rest of the code. Here's a breakdown of each import:

- **`import torch`**: This imports the main torch library, which is the core of PyTorch. PyTorch is an open-source machine learning framework that allows for building and training neural networks.

- **`import torchvision`**: This imports the torchvision library, which is part of the PyTorch project. It provides access to popular datasets, model architectures, and image transformations for computer vision tasks.

- **`from torch import nn`**: This imports the `nn` module from torch. The `nn` module provides a set of building blocks for creating neural networks, such as layers (like convolutional layers, linear layers), activation functions, and loss functions.

- **`from torch.nn import CrossEntropyLoss`**: This specifically imports the `CrossEntropyLoss` class from the `torch.nn` module. This is a common loss function used for classification tasks.

- **`from torch.optim import Adam`**: This imports the Adam optimizer from the `torch.optim` module. Optimizers are algorithms used to update the weights of a neural network during training to minimize the loss function. Adam is a popular and effective optimizer.

- **`from torch.utils.data import DataLoader, Dataset`**: This imports the `DataLoader` and `Dataset` classes from `torch.utils.data`. The `Dataset` class is used to represent a collection of data samples, while the `DataLoader` class provides an efficient way to iterate over the dataset in batches during training.

- **`from torchvision import transforms`**: This imports the `transforms` module from torchvision. The `transforms` module provides a collection of common image transformations that can be applied to images for data augmentation and preprocessing.

- **`from torchvision.datasets import ImageFolder`**: This imports the `ImageFolder` class from `torchvision.datasets`. This class is a convenient way to load image datasets organized in a specific folder structure where the class labels are inferred from the directory names.

- **`from torchvision.models import ResNet152_Weights, resnet152`**: This imports the `resnet152` model architecture and its default weights (`ResNet152_Weights`) from the `torchvision.models` module. ResNet152 is a pre-trained convolutional neural network model that is often used as a starting point for various computer vision tasks.

- **`from tqdm import tqdm`**: This imports the tqdm library. tqdm is a library that provides a smart progress bar for loops, which can be helpful for visualizing the progress of training or other iterative processes.


In [ ]:
import numpy as np
import torch
import torchvision
from torch import nn
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet152_Weights, resnet152
from tqdm import tqdm

In [ ]:
BATCH_SIZE = 64
# LEARNING_RATE = 1e-5
LEARNING_RATE = 5e-6
NO_OF_ITERATIONS = 300
CUDA_DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
PATH_TO_MODEL_SAVE = "resnet_model_weight_for_cnc.pth"

# Image Transformations for Data Augmentation

This section of the code defines a sequence of image transformations that will be applied to the images in the dataset. These transformations are used for data augmentation, which helps to increase the size and variability of the training data. This can improve the generalization ability of the trained model and prevent overfitting. The `transforms.Compose` function is used to chain multiple transformations together to be applied in a specific order.

Here's a breakdown of each transformation:

- **`transforms.Resize((256, 256))`**: This transformation resizes the input image to a fixed size of 256x256 pixels. This is often done to ensure that all images have a consistent size before being fed into a neural network.

- **`transforms.RandomCrop((224, 224))`**: This transformation randomly crops a section of size 224x224 pixels from the resized image. Random cropping introduces variability in the training data by forcing the model to learn features from different parts of the image.

- **`transforms.RandomVerticalFlip(p=0.5)`**: This transformation randomly flips the image vertically with a probability of 0.5.

- **`transforms.RandomHorizontalFlip(p=0.5)`**: This transformation randomly flips the image horizontally with a probability of 0.5. Flipping the image both vertically and horizontally helps the model become invariant to the orientation of objects in the image.

- **`transforms.RandomRotation(degrees=30)`**: This transformation randomly rotates the image by an angle between -30 and +30 degrees. Random rotation helps the model become robust to variations in the rotation of objects.

- **`transforms.ToTensor()`**: This transformation converts the image from a PIL Image or a NumPy array to a PyTorch tensor. It also scales the pixel values to the range [0, 1]. This is necessary because PyTorch models work with tensors.

- **`transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])`**: This transformation normalizes the tensor image with the specified mean and standard deviation for each color channel (Red, Green, Blue). Normalization is a common preprocessing step that helps to standardize the input data and can improve the training process. The values used here are common mean and standard deviation values for images pre-trained on the ImageNet dataset.


In [ ]:
transforms_functions = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=30),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# Creating a Dataset with ImageFolder
This line of code creates a dataset using the ImageFolder class from the torchvision.datasets module.


Here's a breakdown of what's happening:

- **`dataset_of_cnc:`** This is the variable name assigned to the newly created dataset object.

- **`ImageFolder():`** This is the constructor for the ImageFolder class. This class is specifically designed to handle datasets where images are organized in subdirectories, with each subdirectory representing a different class.

- **`root="building_only":`** This argument specifies the root directory where the image dataset is located. In this case, it's expected that your image files are within a folder named "building_only" and that this folder contains subdirectories, where each subdirectory name is the name of a class `(e.g., "building_only/class1", "building_only/class2", etc.).` The ImageFolder class will automatically infer the class labels from these subdirectory names [1].
- **`transform=transforms_functions:`** This argument applies the sequence of image transformations defined earlier in the notebook (stored in the `transforms_functions variable`) to each image as it is loaded from the specified directory. These transformations are used for data augmentation and preprocessing.

In [ ]:
dataset_of_cnc = ImageFolder(root="building_only", transform=transforms_functions)

The ImageFolder class, which was used to create the dataset_of_cnc object, automatically infers the class labels for your dataset based on the names of the subdirectories within the specified root directory [1]. The classes attribute is a list that stores these inferred class names in alphabetical order.

Therefore, executing this line of code will display the names of all the classes that were found in the "building_only" directory when the ImageFolder dataset was created. This is useful for verifying that the ImageFolder correctly identified your desired classes.

In [ ]:
dataset_of_cnc.classes

['Administrative', 'Chemistry', 'Gurudeb', 'Heritage', 'canteen']

# Creating a DataLoader for the Dataset

This line of code creates a DataLoader object, which is used to efficiently load and iterate over the dataset during the training process.


Here's a breakdown of the components:

- **`training_dataloader_for_cnc`**: This is the variable name assigned to the newly created `DataLoader` object.

- **`DataLoader()`**: This is the constructor for the `DataLoader` class from `torch.utils.data`. It's designed to handle the process of loading data from a dataset in batches.

- **`dataset_of_cnc`**: This is the first argument passed to the `DataLoader`. It is the dataset object that was created in a previous step using `ImageFolder`. The `DataLoader` will load data from this dataset.

- **`batch_size=BATCH_SIZE`**: This argument specifies the number of samples (images and their corresponding labels) that will be loaded in each batch. The value for `BATCH_SIZE` is defined earlier in the notebook; in this case, it is 64. Using batches is crucial for efficient training, especially with large datasets.

- **`shuffle=True`**: This argument indicates whether the data should be shuffled at the beginning of each training epoch. When `shuffle` is set to `True`, the `DataLoader` will randomize the order of the samples in the dataset for each pass through the data. This helps to prevent the model from learning the order of the data and improves the generalization of the model.


In [ ]:
training_dataloader_for_cnc = DataLoader(
    dataset_of_cnc, batch_size=BATCH_SIZE, shuffle=True
)

# Initializes a neural network model using the ResNet152 architecture.

ResNet152 is a deep convolutional neural network that is commonly used for image classification tasks.

The code calls the `resnet152()` function from the torchvision.models module. This function creates an instance of the **ResNet152 model.**

- The line # `resnet_model_for_cnc = resnet152(weights=ResNet152_Weights.DEFAULT)` is commented out. If this line were active, it would initialize the ResNet152 model with pre-trained weights. These weights are typically learned from a very large dataset like ImageNet and can provide a good starting point for training on a new dataset, a technique known as transfer learning.

- The active line, `resnet_model_for_cnc = resnet152(),` initializes the ResNet152 model with randomly initialized weights. This means the model will be trained from scratch on your dataset.

The created model is assigned to the variable resnet_model_for_cnc



In [ ]:
# resnet_model_for_cnc = resnet152(weights=ResNet152_Weights.DEFAULT)
resnet_model_for_cnc = resnet152()

This line of code simply displays the structure of the neural network model that was just initialized and assigned to the variable `resnet_model_for_cnc.`



In [ ]:
resnet_model_for_cnc

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Freezing the Model Parameters
This line of code is used to control which parameters of your neural network model, named resnet_model_for_cnc, will be updated during the training process.



Let's clarify the difference between `resnet_model_for_cnc.fc` and `resnet_model_for_cnc`.

- **`resnet_model_for_cnc:`** This refers to the entire neural network model object. It's the complete instance of the ResNet152 architecture that you initialized earlier. Think of this as the whole building that houses all the different parts of the network. When you apply operations or methods directly to `resnet_model_for_cnc`,* you are typically affecting or interacting with the entire model.*

- **`resnet_model_for_cnc.fc:`** This accesses a specific part or module within the `resnet_model_for_cnc` object. The .fc is an attribute of the resnet_model_for_cnc object,* and it specifically points to the *final fully connected layer* (or a sequence of layers in this case, after you redefined it) of the ResNet152 architecture. Think of this as a specific room or section within the building. When you apply operations or methods to` resnet_model_for_cnc`.fc, you are only affecting or interacting with this particular part of the model.

In [ ]:
resnet_model_for_cnc.requires_grad_(requires_grad=False)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Freezing and Unfreezing Model Layers

This section of the code is related to controlling which parts of the neural network model will be updated during the training process. In deep learning, especially when using pre-trained models (although in this case, the model is initialized with random weights), it's common to selectively train only certain layers of the network. This is often done to fine-tune the model for a specific task while keeping the initial learned features from the earlier layers fixed.

The line you are asking about specifically focuses on the final fully connected layer (often named `fc` in many common network architectures like ResNet) of the `resnet_model_for_cnc`.

Let's break this down:

*   **`resnet_model_for_cnc.fc`**: This accesses the fully connected layer of the `resnet_model_for_cnc` object. In a standard ResNet, this is the layer that takes the high-level features extracted by the convolutional layers and maps them to the final class predictions.
*   **`.requires_grad_()`**: This is a method available for PyTorch tensors and modules that allows you to set the `requires_grad` attribute. The underscore at the end of the method name (`_`) indicates that this operation modifies the object in place.
*   **`requires_grad=True`:** This argument sets the `requires_grad` attribute of the parameters within the fully connected layer to `True`.

**What does `requires_grad=True` mean?**

In PyTorch, the `requires_grad` attribute is crucial for the automatic differentiation engine (autograd). When `requires_grad` is set to `True` for a tensor or the parameters within a module, PyTorch will keep track of the operations performed on that tensor or module during the forward pass. This allows PyTorch to compute the gradients of the loss with respect to these parameters during the backward pass.

By setting `resnet_model_for_cnc.fc.requires_grad_` to `True`, you are indicating that the weights and biases of the final fully connected layer **should** be updated during the training process. This means that the optimization algorithm (defined later in the notebook) will calculate the gradients for these parameters and adjust them to minimize the loss function.

In contrast, in the previous code cell, `resnet_model_for_cnc.requires_grad_(requires_grad=False)` was executed. This sets the `requires_grad` attribute to `False` for *all* parameters in the entire `resnet_model_for_cnc` model. This effectively "freezes" the weights of the pre-trained layers (the convolutional and pooling layers before the final fully connected layer), meaning they will not be updated during training.

Therefore, the combined effect of these two lines is to freeze the weights of the majority of the ResNet152 model and only allow the parameters of the final fully connected layer to be trained. This is a common strategy for transfer learning, where you use a pre-trained model as a feature extractor and train a new classification head (the fully connected layer) on your specific dataset. Even when training from scratch, you might selectively freeze layers during initial training phases.

In [ ]:
resnet_model_for_cnc.fc.requires_grad_(requires_grad=True)

Linear(in_features=2048, out_features=1000, bias=True)

# Redefining the Final Layer of the Model

This block of code replaces the original final fully connected layer of the `resnet_model_for_cnc` with a new sequence of layers. This is a common practice when adapting a pre-existing model architecture (like ResNet152) for a specific task, especially when the number of output classes for your task is different from the original task the model was trained on.

Here's a breakdown of what's happening:

- **`resnet_model_for_cnc.fc = ...`**: This line assigns a new module to the `fc` attribute of the `resnet_model_for_cnc` object. As explained previously, the `fc` attribute typically refers to the final fully connected layer responsible for producing the class predictions. By assigning a new sequence of layers here, you are essentially replacing the model's original output layer.

- **`torch.nn.Sequential(...)`**: This is a container in PyTorch that holds a sequence of modules. Data will pass through the modules in the order they are added to the Sequential container.

- **`torch.nn.Linear(in_features=2048, out_features=512)`**: This creates the first new layer, which is a linear (or fully connected) layer. It takes an input of size 2048 (which is the output size of the ResNet's layers before the original `fc` layer) and outputs a tensor of size 512. This layer acts as an intermediate layer in the new classification head.

- **`torch.nn.ReLU()`**: This adds a Rectified Linear Unit (ReLU) activation function after the first linear layer. ReLU introduces non-linearity into the model, which is essential for learning complex patterns in the data.

- **`torch.nn.Dropout(p=0.2)`**: This adds a Dropout layer with a dropout probability of 0.2. Dropout is a regularization technique that randomly sets a fraction of input units to zero during training. This helps prevent the model from overfitting by making it less reliant on specific neurons.

- **`torch.nn.Linear(in_features=512, out_features=len(dataset_of_cnc.classes))`**: This is the second linear layer in the sequence. It takes the output of the previous layers (size 512) and maps it to a tensor with a size equal to the number of classes in your dataset. The `len(dataset_of_cnc.classes)` dynamically gets the number of unique classes identified when the `ImageFolder` dataset was created. This layer is the final output layer of your redefined classification head.

- **`torch.nn.Softmax(dim=1)`**: This adds a Softmax activation function to the output of the final linear layer. Softmax is commonly used in multi-class classification problems. It converts the raw output scores (logits) from the linear layer into probabilities, where each probability represents the likelihood of the input belonging to a specific class. The `dim=1` argument specifies that the Softmax operation should be applied across the class dimension.

In summary, this code replaces the standard final layer of ResNet152 with a new custom classification head consisting of two linear layers, a ReLU activation, and a Dropout layer, followed by a Softmax activation to produce class probabilities. This new head is tailored to the specific number of classes in your dataset.


In [ ]:
resnet_model_for_cnc.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=2048, out_features=512),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.2),
    torch.nn.Linear(
        in_features=512,
        out_features=len(dataset_of_cnc.classes),
    ),
    torch.nn.Softmax(dim=1),
)

# Initializing the Weights and Biases of Linear Layers

This code block iterates through all the modules (layers and sub-modules) within the neural network model `resnet_model_for_cnc` and specifically initializes the weights and biases of any **linear layers** (`nn.Linear`) that it finds. Proper initialization of weights and biases is important for the effective training of neural networks.

Here's a breakdown:

- **`for module in resnet_model_for_cnc.modules():`** This line starts a loop that goes through every `module` within the `resnet_model_for_cnc` model. A module can be a single layer (like a linear layer or a convolutional layer) or a container of other modules (like a `Sequential` module).

- **`if type(module) == nn.Linear:`** Inside the loop, this condition checks if the current `module` is an instance of a `nn.Linear` layer. The code inside this `if` block will only execute for linear layers.

- **` nn.init.kaiming_normal_( module.weight.data, a=0, mode="fan_out", nonlinearity="relu", ):`**
If the module is a linear layer, this line initializes the **weights** of that layer.
  *   `nn.init.kaiming_normal_`: This is a function from PyTorch's initialization module. It implements the Kaiming initialization method (also known as He initialization). This method is particularly well-suited for layers that use the ReLU activation function, which is used in the redefined final layer of the model.
  *   `module.weight.data`: This accesses the weight tensor of the linear layer. The `_` at the end of `kaiming_normal_` indicates that the initialization is performed in-place on the weight tensor's data.
  *   `a=0`, `mode="fan_out"`, `nonlinearity="relu"`: These are parameters for the Kaiming normal initialization, tailored for layers followed by ReLU activation.

- **` if module.bias is not None:`**
 This condition checks if the current linear layer has a **bias** term. By default, `nn.Linear` layers include a bias, but it can be disabled during initialization.


- **`fan_in, fan_out = nn.init.calculate_fan_in_and_fan_out(module.weight.data) bound = 1 / (fan_out) ** 0.5 nn.init.normal(module.bias, -bound, bound)`**:
If the linear layer has a bias, these lines initialize the bias term.
  *   `nn.init._calculate_fan_in_and_fan_out(module.weight.data)`: This internal PyTorch function calculates the number of input and output connections for the linear layer, which is used to determine the scale of the bias initialization.
  *   `bound = 1 / (fan_out) ** 0.5`: This calculates the boundary for the uniform distribution from which the bias values will be drawn. This specific calculation is based on a common practice for initializing biases, often related to Glorot/Xavier or Kaiming initialization schemes.
  *   `nn.init.normal_(module.bias, -bound, bound)`: This initializes the bias tensor (`module.bias`) with values drawn from a uniform distribution between `-bound` and `bound` [1]. The `_` indicates an in-place operation.

In essence, this code ensures that the weights of the linear layers within the model, particularly the newly added layers in the redefined classification head, are initialized using a method suitable for ReLU activations, and the biases are initialized to small random values around zero. This helps the training process converge more effectively.

In [ ]:
for module in resnet_model_for_cnc.modules():
    if type(module) == nn.Linear:
        nn.init.kaiming_normal_(
            module.weight.data,
            a=0,
            mode="fan_out",
            nonlinearity="relu",
        )
        if module.bias is not None:
            fan_in, fan_out = nn.init._calculate_fan_in_and_fan_out(module.weight.data)
            bound = 1 / (fan_out) ** 0.5
            nn.init.normal_(module.bias, -bound, bound)

# Moving the Model to the Specified Device

This line of code is straightforward but essential in PyTorch for managing where your neural network model resides during computations.
The `.to()` method is a general-purpose method in PyTorch for moving objects (like tensors or modules) to a different device. In this case, it's being used on the `resnet_model_for_cnc` object, which represents your neural network.

*   `device=CUDA_DEVICE`: This argument specifies the target device to which you want to move the model. The `CUDA_DEVICE` variable was defined earlier in the notebook based on whether a CUDA-enabled GPU is available.
    *   If a CUDA-enabled GPU is available, `CUDA_DEVICE` will likely be set to a string like `"cuda:1"` (indicating GPU device 1, depending on your system configuration). In this case, the model will be moved to the GPU.
    *   If a CUDA-enabled GPU is **not** available, `CUDA_DEVICE` will be set to `"cpu"`, and the model will be moved to the CPU.

Moving the model to a GPU when available is standard practice in deep learning because GPUs are highly optimized for the parallel computations required for neural network training and inference, leading to significantly faster processing compared to using a CPU.

By executing this line, you ensure that your `resnet_model_for_cnc` is ready to perform calculations on the specified device, which is crucial before starting any training or inference operations.

In [ ]:
resnet_model_for_cnc.to(device=CUDA_DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Loading Model Weights

This line of code is responsible for loading the previously saved weights and biases of the neural network model into the currently initialized `resnet_model_for_cnc`. This is a crucial step if you want to resume training from a checkpoint or use a pre-trained model that you have saved.
Here's a breakdown:

*   `resnet_model_for_cnc.load_state_dict(...)`: This is a method of a PyTorch model (`resnet_model_for_cnc` in this case) that allows you to load a `state_dict`. A `state_dict` in PyTorch is essentially a Python dictionary that maps each layer to its learnable parameters (weights and biases) [2]. By calling `load_state_dict`, you are instructing the model to replace its current parameters with the ones provided in the dictionary.

*   `torch.load(...)`: This function is used to deserialize and load an object that was saved using `torch.save`. In this context, it's loading the saved `state_dict` from a file.

*   `PATH_TO_MODEL_SAVE`: This variable holds the file path where the model's `state_dict` was previously saved. This file contains the learned weights and biases of the model from a prior training session.

*   `map_location=CUDA_DEVICE`: This argument to `torch.load` specifies where the loaded `state_dict` should be placed in memory. The `CUDA_DEVICE` variable determines whether the model should be loaded onto a GPU (`cuda:1` if available) or the CPU (`cpu`). This is important to ensure that the loaded weights are in the correct device memory for subsequent operations.

**In summary:** This line of code loads the saved weights and biases of the model from the file specified by `PATH_TO_MODEL_SAVE` and applies them to the `resnet_model_for_cnc` object. This allows the model to start training or inference from a previously saved state [1].

In [ ]:
resnet_model_for_cnc.load_state_dict(
    torch.load(PATH_TO_MODEL_SAVE, map_location=CUDA_DEVICE)
)

<All keys matched successfully>

In [ ]:
torch.cuda.is_available()

True

# Defining the Optimizer

This line of code initializes an optimizer, which is a crucial component in the training process of a neural network. The optimizer's role is to update the model's parameters (weights and biases) during training in a way that minimizes the loss function.
Here's a breakdown:

*   `optimizer`: This is the variable name assigned to the optimizer object that is being created.
*   `Adam(...)`: This is the constructor for the **Adam optimizer**, imported from `torch.optim`. Adam is a popular and effective optimization algorithm that adapts the learning rate for each parameter individually based on estimates of first and second moments of the gradients.
*   `resnet_model_for_cnc.parameters()`: This argument provides the optimizer with the parameters (weights and biases) of the neural network model (`resnet_model_for_cnc`) that it needs to optimize. The `.parameters()` method is a standard way in PyTorch to access all the learnable parameters of a model.
*   `lr=LEARNING_RATE`: This argument sets the **learning rate** for the optimizer. The learning rate is a hyperparameter that determines the step size at each iteration while moving towards a minimum of the loss function. A smaller learning rate means smaller updates, and a larger learning rate means larger updates. The value for `LEARNING_RATE` is defined earlier in the notebook as `5e-6`. Choosing an appropriate learning rate is important for successful training.

In essence, this line sets up the Adam optimizer to be used for training the `resnet_model_for_cnc`. It specifies which parameters of the model should be updated and the initial learning rate to be used by the optimizer.

In [ ]:
optimizer = Adam(resnet_model_for_cnc.parameters(), lr=LEARNING_RATE)

# Defining the Loss Function

Here's what it does:

- **`loss_function`**: This is the variable name you are assigning to the created loss function object. You can use this variable later in your code to calculate the loss.

- **`CrossEntropyLoss()`**: This is the constructor for the `CrossEntropyLoss` class, which was imported from the `torch.nn` module earlier in the notebook. The `CrossEntropyLoss` is a very common and widely used loss function for classification problems. In the context of training a neural network for classification (like the image classification task this code is likely for), the loss function measures the difference between the model's predicted output and the actual target label.

Specifically, `CrossEntropyLoss` is often used when the model outputs raw, unnormalized scores (sometimes called logits) for each class, and the target labels are integers representing the correct class index. The `CrossEntropyLoss` combines two operations:

1. **Softmax**: It applies a Softmax function to the model's output logits to convert them into probabilities across all classes. These probabilities represent the model's confidence that the input belongs to each class.

2. **Negative Log Likelihood (NLL) Loss**: It then calculates the negative log likelihood of the true class given the predicted probabilities. This measures how "surprised" the model is by the correct answer. A lower negative log likelihood means the model assigned a high probability to the correct class, resulting in a lower loss.

By using `CrossEntropyLoss`, the training process aims to adjust the model's parameters so that it assigns higher probabilities to the correct classes for the training data, thereby minimizing this loss function. This leads to the model learning to correctly classify the input images.

In simpler terms, `loss_function = CrossEntropyLoss()` sets up the mechanism to tell your model how wrong it is during training, specifically for a task where you are trying to categorize inputs into different classes. The optimizer then uses this loss value to figure out how to adjust the model to make it less wrong in the future.


In [ ]:
loss_function = CrossEntropyLoss()

# The Training Loop

This block of code represents the core training loop for your neural network model. It iterates through the dataset multiple times (epochs) to train the model.

Let's break down the code step by step:
- **`for i_iteration in tqdm(range(NO_OF_ITERATIONS)):`**

This is the outer loop that iterates for a specified number of **iterations** (or epochs). The number of iterations is determined by the value of the `NO_OF_ITERATIONS` variable, which was set earlier in the notebook to 300. The `tqdm` function wraps the `range` object, providing a progress bar that shows the progress of this loop, making it easy to track how many iterations have been completed.
- **`training_loss = []`**
    **`for i_image, i_target in dataloader_for_cnc:`**

Inside the main iteration loop, this starts a nested loop. This inner loop iterates over batches of data provided by the `dataloader_for_cnc`. The `dataloader_for_cnc` is configured to load the images and their corresponding labels in batches of size `BATCH_SIZE` (64). In each iteration of this inner loop, `i_image` will contain a batch of images, and `i_target` will contain the corresponding batch of labels. An empty list `training_loss` is initialized at the beginning of each outer iteration to store the loss for each batch within that iteration.
- **`i_image = i_image.to(device=CUDA_DEVICE) \
i_target = i_target.to(device=CUDA_DEVICE)`**

These lines move the current batch of images (`i_image`) and their target labels (`i_target`) to the specified device, which is determined by the `CUDA_DEVICE` variable. This ensures that the data is on the same device as the model before computations are performed, which is essential for efficient training, especially when using a GPU.
- **`optimizer.zero_grad()`**  

Before performing the forward pass and calculating gradients, it's crucial to clear any previously computed gradients. This line calls the `zero_grad()` method of the optimizer, which sets the gradients of all optimized parameters to zero. This prevents gradients from accumulating across batches.
- **`output = resnet_model_for_cnc(i_image)`**  

This line performs the **forward pass**. The batch of images (`i_image`) is passed through the neural network model (`resnet_model_for_cnc`). The model processes the images and produces an `output` tensor, which represents the model's predictions for each image in the batch.
- **`loss = loss_function(output, i_target)`**  

This line calculates the **loss**. The `loss_function` (which was defined as `CrossEntropyLoss` earlier) compares the model's predictions (`output`) with the actual target labels (`i_target`) and calculates a measure of how well the model is performing. The lower the loss, the better the model's predictions match the true labels [1].
- **`loss.backward()`**  

This line performs the **backward pass**. Based on the calculated `loss`, PyTorch's automatic differentiation engine computes the gradients of the loss with respect to the model's parameters. These gradients indicate how much each parameter should be adjusted to reduce the loss.
- **`optimizer.step()`**  

This line updates the model's parameters. The `optimizer` uses the gradients computed during the backward pass to adjust the weights and biases of the model. The `step()` method applies the chosen optimization algorithm (Adam in this case) to update the parameters in a way that minimizes the loss function.
- **`training_loss.append(loss.item())`**  

The loss for the current batch is extracted as a Python number using `.item()` and appended to the `training_loss` list. This list accumulates the loss for each batch within the current iteration.
- **`print("Training loss at", i_iteration, "iteration is", np.mean(training_loss))`**  

After processing all batches in an iteration (the inner loop finishes), this line calculates the average training loss for that iteration by taking the mean of the losses stored in the `training_loss` list. It then prints the iteration number and the calculated average training loss. This provides feedback on the model's training progress.
- **`torch.save(resnet_model_for_cnc.state_dict(), PATH_TO_MODEL_SAVE)`**  

Finally, after each iteration, this line saves the current state of the model's parameters (weights and biases). The `state_dict()` method returns a dictionary containing the learnable parameters of the model [2]. This dictionary is then saved to a file specified by `PATH_TO_MODEL_SAVE`. Saving the model's state periodically allows you to resume training from a checkpoint or use the trained model later for inference.

In [ ]:
for i_iteration in tqdm(range(NO_OF_ITERATIONS)):
    training_loss = []
    for i_image, i_target in dataloader_for_cnc:
        i_image = i_image.to(device=CUDA_DEVICE)
        i_target = i_target.to(device=CUDA_DEVICE)
        optimizer.zero_grad()

        output = resnet_model_for_cnc(i_image)
        loss = loss_function(output, i_target)
        loss.backward()
        optimizer.step()

        training_loss.append(loss.item())

    print("Training loss at", i_iteration, "iteration is", np.mean(training_loss))

    torch.save(resnet_model_for_cnc.state_dict(), PATH_TO_MODEL_SAVE)

  0%|                                                   | 0/300 [00:00<?, ?it/s]

Training loss at 0 iteration is 0.9054904466583615


  0%|▏                                        | 1/300 [00:24<2:00:50, 24.25s/it]

Training loss at 1 iteration is 0.9049198882920402


  1%|▎                                        | 2/300 [00:48<2:00:06, 24.18s/it]

...

 99%|██████████████████████████████████████▋| 298/300 [2:00:31<00:48, 24.32s/it]

Training loss at 298 iteration is 0.9056140354701451


100%|██████████████████████████████████████▊| 299/300 [2:00:56<00:24, 24.48s/it]

Training loss at 299 iteration is 0.9056733335767474


100%|███████████████████████████████████████| 300/300 [2:01:20<00:00, 24.27s/it]
